### Retrieve global coins blockchain data

In [169]:
import time, datetime
import requests
import pandas as pd
pd.set_option('display.precision', 10)

api_token = '8a0b92957b57d4eea9cdf73c4d7436c09c82a38865b84e5b2304b7b5a0202b92'
coins_endpoint = f'https://whattomine.com/api/v1/coins?api_token={api_token}'
coins_data = requests.get(coins_endpoint).json()

coins = [ 'BTC', 'BCH', 'DOGE', 'LTC', 'KAS', 'RVN' ]
columns = [ 'block_time', 'block_reward', 'exchange_rate_btc', 'algorithm', 'nethash_eh_s' ] 
index = []
data = []
btc_exchange_rate = 0

def get_exchange_rate(tag, price):
    if tag == 'BTC':
        global btc_exchange_rate
        btc_exchange_rate = price
        return 1
    return price

for c in coins_data:
    if c['tag'] not in coins:
        continue
    index.append(c['tag'])
    data.append([ float(c['block_time']), c['block_reward'],  get_exchange_rate(c['tag'], c['exchanges'][0]['price']), c['algorithm'], c['nethash'] / 1e18 ])

global_df = pd.DataFrame(data, index=index, columns=columns)
global_df['blocks_per_day'] = 86400 / global_df.block_time
global_df['block_reward_btc'] = global_df.block_reward * global_df.exchange_rate_btc
global_df['daily_emmisions_btc'] = global_df.blocks_per_day * global_df.block_reward_btc
global_df['index_price_btc_eh_day'] = global_df.daily_emmisions_btc / global_df.nethash_eh_s # BTC / EH / day
# global_df.loc['DOGE+LTC', ['daily_emmisions_btc']] = global_df.loc['DOGE', ['daily_emmisions_btc']] + global_df.loc['LTC', ['daily_emmisions_btc']]
print('Global blockchain data:')
# print(df)
global_df

Global blockchain data:


,block_time,block_reward,exchange_rate_btc,algorithm,nethash_eh_s,blocks_per_day,block_reward_btc,daily_emmisions_btc,index_price_btc_eh_day
BCH,613.000,3.1250000000,0.0033295067,SHA-256,2.6948384075,140.9461663948,0.0104047083,1.4665037500,0.5441898653
BTC,650.000,3.1695690500,1.0000000000,SHA-256,754.3764523085,132.9230769231,3.1695690500,421.3088706462,0.5584862430
DOGE,64.000,10000.0000000000,0.0000026550,Scrypt,0.0016329000,1350.0000000000,0.0265500811,35.8426094189,21950.2788562250
KAS,1.001,65.5729307474,0.0000011212,kHeavyHash,1.2557675347,86313.6863136863,0.0000735189,6.3456865043,5.0532334441
LTC,163.000,6.2500000000,0.0014190000,Scrypt,0.0020896573,530.0613496933,0.0088687500,4.7009815951,2249.6423520706
RVN,60.000,2500.0000000000,0.0000001483,KawPow,0.0000066832,1440.0000000000,0.0003706631,0.5337548242,79865.0341535240


### Retrieve NiceHash marketplace algorithms data

In [170]:
import requests
import pandas as pd

algorithms_response = requests.get('https://api2.nicehash.com/main/api/v2/public/simplemultialgo/info')
multialgo_data = algorithms_response.json()
algorithms = [ 'SCRYPT', 'SHA256ASICBOOST', 'KAWPOW', 'KHEAVYHASH']
columns = ['paying', 'speed']
index = []
data = []
for a in multialgo_data['miningAlgorithms']:
    algo = a['algorithm']
    if algo not in algorithms:
        continue
    index.append(algo)
    data.append([a['paying'], a['speed']])
algo_data_df = pd.DataFrame(data, index=index, columns=columns)

factors_response = requests.get('https://api2.nicehash.com/main/api/v2/mining/algorithms')
factors_data = factors_response.json()
columns = ['market_factor', 'mining_factor', 'price_factor']
index = []
data = []
for f in factors_data['miningAlgorithms']:
    algo = f['algorithm']
    if algo not in algorithms:
        continue
    index.append(algo)
    data.append([float(f['marketFactor']), float(f['miningFactor']), float(f['priceFactor'])])
factors_data_df = pd.DataFrame(data, index=index, columns=columns)
factors_data_df

sats2btc = 1e8
nh_market_df = pd.DataFrame(index=index)
nh_market_df['hashrate_eh_s'] = algo_data_df['speed'] * factors_data_df['mining_factor'] / 1e18 # EH / s
nh_market_df['paying_btc_eh_day'] = algo_data_df['paying'] / sats2btc  * 1e18 # BTC/EH/day
nh_market_df['paying_btc_eh_day_$'] = (nh_market_df.paying_btc_eh_day * btc_exchange_rate).map('${:,.2f}'.format)
nh_market_df['daily_consumption_btc'] = nh_market_df.hashrate_eh_s * nh_market_df.paying_btc_eh_day
print('Nicehash marketplace data:')
# print(market_data_df)
nh_market_df

Nicehash marketplace data:


,hashrate_eh_s,paying_btc_eh_day,paying_btc_eh_day_$,daily_consumption_btc
SCRYPT,0.0001061102,17644.4682921635,"$1,680,577,201.64",1.8722581755
SHA256ASICBOOST,4.2560440903,0.5500588282,"$52,391.28",2.3410746250
KAWPOW,0.0000007138,75640.0719862313,"$7,204,466,488.05",0.0539925660
KHEAVYHASH,0.0470348351,4.8929405435,"$466,036.39",0.2301386517


### Calculate Nicehash max daily and max hourly consumption rate

In [171]:
max_market_utilisation = 0.75

nh_limits_df = pd.DataFrame(index=nh_market_df.index)
nh_limits_df['max_hashrate_eh_s'] = nh_market_df.hashrate_eh_s * max_market_utilisation
nh_limits_df['max_daily_consumption_btc'] = nh_market_df.daily_consumption_btc * max_market_utilisation
nh_limits_df['max_hourly_consumption_btc'] = nh_limits_df.max_daily_consumption_btc / 24
nh_limits_df['max_hourly_consumption_$'] = (nh_limits_df.max_hourly_consumption_btc * btc_exchange_rate).map('${:,.2f}'.format)
nh_limits_df

,max_hashrate_eh_s,max_daily_consumption_btc,max_hourly_consumption_btc,max_hourly_consumption_$
SCRYPT,0.0000795827,1.4041936317,0.0585080680,"$5,572.70"
SHA256ASICBOOST,3.1920330677,1.7558059688,0.0731585820,"$6,968.11"
KAWPOW,0.0000005354,0.0404944245,0.0016872677,$160.71
KHEAVYHASH,0.0352761263,0.1726039888,0.0071918329,$685.00


### Calculate Nicehash hashrate to global hashrate ratio
Cap Nicehash hashrate to 15% of global hashrate

In [172]:
import numpy as np

max_allowed_coin_market_hp_share = 0.15

reachable_df = pd.DataFrame(index=global_df.index)
reachable_df['algorithm'] = global_df['algorithm']
reachable_df['nethash_eh_s'] = global_df['nethash_eh_s']
reachable_df.loc['BCH', 'nh_hashrate_eh_s'] = nh_limits_df.loc['SHA256ASICBOOST', 'max_hashrate_eh_s']
reachable_df.loc['BTC', 'nh_hashrate_eh_s'] = nh_limits_df.loc['SHA256ASICBOOST', 'max_hashrate_eh_s']
reachable_df.loc['DOGE', 'nh_hashrate_eh_s'] = nh_limits_df.loc['SCRYPT', 'max_hashrate_eh_s']
reachable_df.loc['LTC', 'nh_hashrate_eh_s'] = nh_limits_df.loc['SCRYPT', 'max_hashrate_eh_s']
reachable_df.loc['KAS', 'nh_hashrate_eh_s'] = nh_limits_df.loc['KHEAVYHASH', 'max_hashrate_eh_s']
reachable_df.loc['RVN', 'nh_hashrate_eh_s'] = nh_limits_df.loc['KAWPOW', 'max_hashrate_eh_s']
reachable_df['nh_to_global_ratio'] = reachable_df['nh_hashrate_eh_s'] / reachable_df['nethash_eh_s']  
reachable_df['nh_to_global_ratio_%'] = reachable_df['nh_to_global_ratio'] * 100
reachable_df['nh_hashrate_eh_s_capped'] = np.minimum(reachable_df.nh_to_global_ratio, max_allowed_coin_market_hp_share) * reachable_df['nethash_eh_s']
reachable_df['nh_to_global_ratio_capped'] = reachable_df['nh_hashrate_eh_s_capped'] / reachable_df['nethash_eh_s']  

reachable_df

,algorithm,nethash_eh_s,nh_hashrate_eh_s,nh_to_global_ratio,nh_to_global_ratio_%,nh_hashrate_eh_s_capped,nh_to_global_ratio_capped
BCH,SHA-256,2.6948384075,3.1920330677,1.1844988771,118.4498877126,0.4042257611,0.1500000000
BTC,SHA-256,754.3764523085,3.1920330677,0.0042313530,0.4231353004,3.1920330677,0.0042313530
DOGE,Scrypt,0.0016329000,0.0000795827,0.0487370059,4.8737005936,0.0000795827,0.0487370059
KAS,kHeavyHash,1.2557675347,0.0352761263,0.0280912871,2.8091287089,0.0352761263,0.0280912871
LTC,Scrypt,0.0020896573,0.0000795827,0.0380840698,3.8084069766,0.0000795827,0.0380840698
RVN,KawPow,0.0000066832,0.0000005354,0.0801047309,8.0104730936,0.0000005354,0.0801047309


### Calculate burn rate needed for specific amount to be spent in the time period
```
B - burn rate
A - amount
P - price
D - duration

B = A / P * D
```

In [173]:
min_order_amount_btc = 0.0001 # BTC
order_duration_s = 3600  # 1 hour

speed_df = pd.DataFrame(index=nh_market_df.index)
speed_df['amount_btc'] = min_order_amount_btc
speed_df['duration_s'] = order_duration_s
speed_df['paying_btc_eh_s'] = nh_market_df['paying_btc_eh_day'] / 86400
speed_df['speed_eh_s'] = speed_df.amount_btc / (speed_df.paying_btc_eh_s * speed_df.duration_s)
speed_df

,amount_btc,duration_s,paying_btc_eh_s,speed_eh_s
SCRYPT,0.0001,3600,0.2042183830,0.0000001360
SHA256ASICBOOST,0.0001,3600,0.0000063664,0.0043631697
KAWPOW,0.0001,3600,0.8754637961,0.0000000317
KHEAVYHASH,0.0001,3600,0.0000566313,0.0004905026


### Calculate amount needed to buy speed for the time period
```
B - burn rate
A - amount
P - price
D - duration

A = B * P * D 
```

In [174]:
speed_eh_s = 1 # EH / s
order_duration_s = 3600  # 1 hour

amount_df = pd.DataFrame(index=nh_market_df.index)
amount_df['speed_eh_s'] = speed_eh_s
amount_df['duration_s'] = order_duration_s
amount_df['paying_btc_eh_s'] = nh_market_df['paying_btc_eh_day'] / 86400
amount_df['amount_btc'] = amount_df.speed_eh_s * amount_df.paying_btc_eh_s * amount_df.duration_s
amount_df

,speed_eh_s,duration_s,paying_btc_eh_s,amount_btc
SCRYPT,1,3600,0.2042183830,735.1861788401
SHA256ASICBOOST,1,3600,0.0000063664,0.0229191178
KAWPOW,1,3600,0.8754637961,3151.6696660930
KHEAVYHASH,1,3600,0.0000566313,0.2038725226


### Calculate RTP for the coin on the NiceHash market

In [175]:
import pandas as pd
import numpy as np

max_allowed_coin_market_hp_share = 0.15
max_nicehash_market_utilisation = 0.76
speed_eh_s = 1
order_duration_s = 3600
order_amount_btc = 1

rtp_df = pd.DataFrame()
# rtp_df.loc['BCH', 'nh_hashrate_eh_s'] = np.minimum(nh_market_df.loc['SHA256ASICBOOST', 'hashrate_eh_s'] * max_nicehash_market_utilisation, global_df.loc['BCH', 'nethash_eh_s'] * max_allowed_coin_market_hp_share)
# rtp_df.loc['BTC', 'nh_hashrate_eh_s'] = np.minimum(nh_market_df.loc['SHA256ASICBOOST', 'hashrate_eh_s'] * max_nicehash_market_utilisation, global_df.loc['BTC', 'nethash_eh_s'] * max_allowed_coin_market_hp_share)
# rtp_df.loc['DOGE', 'nh_hashrate_eh_s'] = np.minimum(nh_market_df.loc['SCRYPT', 'hashrate_eh_s'] * max_nicehash_market_utilisation, global_df.loc['DOGE', 'nethash_eh_s'] * max_allowed_coin_market_hp_share)
# rtp_df.loc['LTC', 'nh_hashrate_eh_s'] = np.minimum(nh_market_df.loc['SCRYPT', 'hashrate_eh_s'] * max_nicehash_market_utilisation, global_df.loc['LTC', 'nethash_eh_s'] * max_allowed_coin_market_hp_share)
# rtp_df.loc['KAS', 'nh_hashrate_eh_s'] = np.minimum(nh_market_df.loc['KHEAVYHASH', 'hashrate_eh_s'] * max_nicehash_market_utilisation, global_df.loc['KAS', 'nethash_eh_s'] * max_allowed_coin_market_hp_share)
# rtp_df.loc['RVN', 'nh_hashrate_eh_s'] = np.minimum(nh_market_df.loc['KAWPOW', 'hashrate_eh_s'] * max_nicehash_market_utilisation, global_df.loc['RVN', 'nethash_eh_s'] * max_allowed_coin_market_hp_share)
# rtp_df['nh_to_global_ratio'] = rtp_df['nh_hashrate_eh_s'] / global_df['nethash_eh_s']
# rtp_df['max_global_btc_reward_h'] = rtp_df.nh_to_global_ratio * global_df['daily_emmisions_btc'] / 24
# rtp_df['max_global_btc_reward_h'] = global_df['daily_emmisions_btc'] / 24
rtp_df['global_price_btc_eh_s'] = global_df['index_price_btc_eh_day'] / 86400
rtp_df.loc['BCH', 'nh_price_btc_eh_s'] = nh_market_df.loc['SHA256ASICBOOST', 'paying_btc_eh_day'] / 86400
rtp_df.loc['BTC', 'nh_price_btc_eh_s'] = nh_market_df.loc['SHA256ASICBOOST', 'paying_btc_eh_day'] / 86400
rtp_df.loc['DOGE', 'nh_price_btc_eh_s'] = nh_market_df.loc['SCRYPT', 'paying_btc_eh_day'] / 86400
# rtp_df.loc['LTC', 'nh_price_btc_eh_s'] = nh_market_df.loc['SCRYPT', 'paying_btc_eh_day'] / 86400
# rtp_df.loc['KAS', 'nh_price_btc_eh_s'] = nh_market_df.loc['KHEAVYHASH', 'paying_btc_eh_day'] / 86400
# rtp_df.loc['RVN', 'nh_price_btc_eh_s'] = nh_market_df.loc['KAWPOW', 'paying_btc_eh_day'] / 86400
rtp_df['global_1eh_1h_duration_reward_btc'] = rtp_df.global_price_btc_eh_s * speed_eh_s * order_duration_s
rtp_df['nh_1eh_1h_duration_expense_btc'] = rtp_df.nh_price_btc_eh_s * speed_eh_s * order_duration_s
rtp_df['rtp_%'] =  rtp_df.global_1eh_1h_duration_reward_btc / rtp_df.nh_1eh_1h_duration_expense_btc * 100
rtp_df['allocation_%'] = rtp_df['rtp_%'] / rtp_df['rtp_%'].sum() * 100

# timestamp = datetime.datetime.fromtimestamp(time.time()).isoformat()
# rtp_df.to_csv(f'RTP - {timestamp}.csv')

rtp_df


,global_price_btc_eh_s,nh_price_btc_eh_s,global_1eh_1h_duration_reward_btc,nh_1eh_1h_duration_expense_btc,rtp_%,allocation_%
BCH,0.0000062985,0.0000063664,0.0226745777,0.0229191178,98.9330299646,30.4532730096
BTC,0.0000064640,0.0000063664,0.0232702601,0.0229191178,101.5320933624,31.2533090264
DOGE,0.2540541534,0.2042183830,914.5949523427,735.1861788401,124.4031755039,38.2934179640
KAS,0.0000584865,NaN,0.2105513935,NaN,NaN,NaN
LTC,0.0260375272,NaN,93.7350980029,NaN,NaN,NaN
RVN,0.9243638212,NaN,3327.7097563968,NaN,NaN,NaN
